### Query an Ensemle model with Triton on http/REST

In [1]:
!pip -q install tritonclient['http']


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports and parameters (adapt to your endpoint and model)
import numpy as np
import tritonclient.http as httpclient
import subprocess
import gevent.ssl

host = "ensemble-rest-verbose-kserve-triton-ensemble.apps.cluster-kgqjf.dynamic.redhatworkshops.io"
port = 443
ssl_connection = True
ssl_context_factory = gevent.ssl._create_unverified_context
model_name = 'ensemble_merger_google_xgb'

In [3]:
# Initialize client
triton_client = httpclient.InferenceServerClient(
                url=host,
                ssl=ssl_connection,
                insecure=True,
                verbose=False,
                ssl_context_factory=ssl_context_factory,
            )

In [4]:
# Prepare input and output objects
inputs = []
outputs = []
inputs.append(httpclient.InferInput("INPUT", [1, 63], "FP32"))
outputs.append(httpclient.InferRequestedOutput("OUTPUT", binary_data=False))

# Create an array of 63 random floats between 0 and 1
input_data = np.arange(start=0, stop=63, dtype=np.float32)

# Expand the input into a batch (size=1)
input_data = np.expand_dims(input_data, axis=0)

# Initialize the data
inputs[0].set_data_from_numpy(input_data, binary_data=False)

In [5]:
# Make the inference request
results = triton_client.infer(
        model_name,
        inputs,
        outputs=outputs
    )

In [6]:
# Get and display results
output_data = results.as_numpy("OUTPUT")
output_data

array([[-2.8066086e+03,  2.8076086e+03,  1.0000000e+00,  0.0000000e+00,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00]], dtype=float32)

In [7]:
# Get and display statistics from the server
statistics = triton_client.get_inference_statistics(model_name=model_name)
print(statistics)

{'model_stats': [{'name': 'ensemble_merger_google_xgb', 'version': '1', 'last_inference': 1709763613455, 'inference_count': 5, 'execution_count': 5, 'inference_stats': {'success': {'count': 5, 'ns': 102309599}, 'fail': {'count': 0, 'ns': 0}, 'queue': {'count': 5, 'ns': 51358}, 'compute_input': {'count': 5, 'ns': 905599}, 'compute_infer': {'count': 5, 'ns': 92666425}, 'compute_output': {'count': 5, 'ns': 2270472}, 'cache_hit': {'count': 0, 'ns': 0}, 'cache_miss': {'count': 0, 'ns': 0}}, 'batch_stats': [{'batch_size': 1, 'compute_input': {'count': 5, 'ns': 905599}, 'compute_infer': {'count': 5, 'ns': 92666425}, 'compute_output': {'count': 5, 'ns': 2270472}}], 'memory_usage': []}]}
